In [1]:
import sys
sys.path.append('..')
print(sys.path)

['c:\\Users\\terc.vinicius\\covid19datascience\\notebooks', 'c:\\Users\\terc.vinicius\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles\\vscode_datascience_helpers\\..\\.does-not-exist', 'c:\\Users\\terc.vinicius\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles', 'c:\\Users\\terc.vinicius\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles\\lib\\python', 'c:\\programdata\\anaconda3\\python38.zip', 'c:\\programdata\\anaconda3\\DLLs', 'c:\\programdata\\anaconda3\\lib', 'c:\\programdata\\anaconda3', '', 'C:\\Users\\terc.vinicius\\AppData\\Roaming\\Python\\Python38\\site-packages', 'c:\\programdata\\anaconda3\\lib\\site-packages', 'c:\\programdata\\anaconda3\\lib\\site-packages\\win32', 'c:\\programdata\\anaconda3\\lib\\site-packages\\win32\\lib', 'c:\\programdata\\anaconda3\\lib\\site-packages\\Pythonwin', 'c:\\programdata\\anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\terc.vinicius\\.ipython', '..']


In [2]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256
import codecs


from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [3]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0


797.8060698509216

In [4]:
# casos = casos_confirmados.fix_mun_resid_casos('mun_atend')
# obitos = casos_confirmados.fix_mun_resid_obitos()
casos = casos_confirmados.get_casos()
casos = casos.drop(columns=['obito','data_obito','excluir'])
obitos = casos_confirmados.get_obitos()

print(f"{len(casos)};{len(obitos)}")
obitos.loc[~obitos['hash'].isin(casos['hash'])]

890682;19531


,ordem,ibge7,nome,sexo,idade,municipio,rs,data_do_obito,excluir,hash,hash_less,hash_more,hash_dt_obito
18989,18998,4106902,LEONI DE BARROS ALBIGAUS,F,77,CURITIBA,02,2020-12-22,NaN,LEONIDEBARROSALBIGAUS77CURITIBA,LEONIDEBARROSALBIGAUS76CURITIBA,LEONIDEBARROSALBIGAUS78CURITIBA,LEONIDEBARROSALBIGAUS22122020


In [5]:
obitos.loc[obitos['hash'].isin(casos['hash_less']),'idade'] += 1
obitos.loc[obitos['hash'].isin(casos['hash_less']),'hash'] = obitos.loc[obitos['hash'].isin(casos['hash_less'])].apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['municipio']), axis=1)

In [6]:
obitos.loc[obitos['hash'].isin(casos['hash_more']),'idade'] -= 1
obitos.loc[obitos['hash'].isin(casos['hash_more']),'hash'] = obitos.loc[obitos['hash'].isin(casos['hash_more'])].apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['municipio']), axis=1)

In [7]:
obitos.loc[~obitos['hash'].isin(casos['hash'])]

,ordem,ibge7,nome,sexo,idade,municipio,rs,data_do_obito,excluir,hash,hash_less,hash_more,hash_dt_obito
18989,18998,4106902,LEONI DE BARROS ALBIGAUS,F,77,CURITIBA,02,2020-12-22,NaN,LEONIDEBARROSALBIGAUS77CURITIBA,LEONIDEBARROSALBIGAUS76CURITIBA,LEONIDEBARROSALBIGAUS78CURITIBA,LEONIDEBARROSALBIGAUS22122020


In [8]:
obitos = obitos.rename(columns={'data_do_obito': 'data_obito'})
obitos['obito'] = 'SIM'

casos = pd.merge(left=casos, right=obitos[['obito','data_obito','hash']], on='hash', how='left')

casos.loc[casos['obito'].isnull(), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'data_obito'] = pd.NaT

print(f"{len(casos)};{len(casos.loc[casos['obito']=='SIM'])}")

890682;19530


In [9]:
# def get_better(cc):
#     scores = np.zeros(len(cc))

#     for i, (_, cc_row) in enumerate(cc.iterrows()):
#         # if cc_row['obito'] == 'SIM':
#         scores[i] = i

#     i = np.argmax(scores)
#     return cc.iloc[i].name

In [10]:
# identificacao = pd.read_excel(join('input','id_access.xlsx'), dtype={'ordem':'str'}).sort_values('ordem')
# casos = pd.merge(casos,identificacao,how='left',on='ordem')

In [11]:
# casos['nome_hash'] = casos['nome'].apply(normalize_hash)
# new_casos = casos.sort_values('nome_hash')[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','nome_hash','hash_dt_diag']]

# obitos['nome_hash'] = obitos['nome'].apply(normalize_hash)
# new_obitos = obitos.sort_values('nome_hash')[['ordem','ibge7','nome','sexo','idade','municipio','rs','data_obito','nome_hash','hash_dt_obito']]

In [12]:
# casos_duplicados = new_casos.loc[new_casos.duplicated('hash_dt_diag',keep=False)].sort_values('nome_hash')

# writer = pd.ExcelWriter(join(output,'casos_duplicados.xlsx'),
#                         engine='xlsxwriter',
#                         datetime_format='dd/mm/yyyy',
#                         date_format='dd/mm/yyyy')

# casos_duplicados.to_excel(writer,sheet_name="casos_duplicados",index=None)
# worksheet = writer.sheets["casos_duplicados"]
# auto_fit_columns(worksheet,casos_duplicados)

# writer.save()
# writer.close()

In [13]:
# casos['excluir'] = 'NAO'
# manter = []
# casos_duplicados = casos_duplicados.sort_values('ordem')

# for cc_group in casos_duplicados.groupby('hash_dt_diag'):
    
#     idx = get_better(cc_group[1])
#     manter.append(idx)

# duplicados = set(casos_duplicados.index.tolist())
# duplicados = duplicados - set(manter)

# # casos.loc[duplicados,'excluir'] = 'SIM'

In [14]:
# obitos_duplicados = new_obitos.loc[new_obitos.duplicated('hash_dt_obito',keep=False)].sort_values('nome_hash')

# writer = pd.ExcelWriter(join(output,'obitos_duplicados.xlsx'),
#                         engine='xlsxwriter',
#                         datetime_format='dd/mm/yyyy',
#                         date_format='dd/mm/yyyy')

# obitos_duplicados.to_excel(writer,sheet_name="obitos_duplicados",index=None)
# worksheet = writer.sheets["obitos_duplicados"]
# auto_fit_columns(worksheet,obitos_duplicados)

# writer.save()
# writer.close()

In [15]:
obitos['excluir'] = None
casos['excluir'] = None

In [16]:
new_casos = casos[['ordem','identificacao','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir']]
new_obitos = obitos[['ordem','ibge7','nome','sexo','idade','municipio','rs','data_obito','excluir']]
new_obitos.shape

(19531, 9)

In [17]:
writer = pd.ExcelWriter(join('..','input','casos_obitos.xlsx'),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos = new_casos.rename(columns={'ibge7':'ibge'})
new_casos.loc[new_casos['rs'].notnull()][['ibge','dt_diag']].to_excel(writer,'casos', index=False)

new_obitos = new_obitos.rename(columns={'ibge7':'ibge'})
new_obitos.loc[new_obitos['rs'].notnull()][['ibge','data_obito']].to_excel(writer,'obitos', index=False)

writer.save()
writer.close()

FileCreateError: [Errno 2] No such file or directory: 'input\\casos_obitos.xlsx'

In [17]:
writer = pd.ExcelWriter(join(output,f"Casos Confirmados Regionais.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

for grupo_casos, grupo_obitos in zip(new_casos.groupby('rs'),new_obitos.groupby('rs')):
    grupo_casos[1].to_excel(writer,sheet_name=f"casos_rs{grupo_casos[0]}",index=None)
    grupo_obitos[1].to_excel(writer,sheet_name=f"obitos_rs{grupo_casos[0]}",index=None)
    worksheet = writer.sheets[f"casos_rs{grupo_casos[0]}"]
    auto_fit_columns(worksheet,grupo_casos[1])
    worksheet = writer.sheets[f"obitos_rs{grupo_casos[0]}"]
    auto_fit_columns(worksheet,grupo_obitos[1])

new_casos.loc[new_casos['rs'].isnull()].to_excel(writer,sheet_name='casos_fora',index=None)
new_obitos.loc[new_obitos['rs'].isnull()].to_excel(writer,sheet_name='obitos_fora',index=None)

writer.save()
writer.close()

In [18]:
writer = pd.ExcelWriter(join(output,"Casos confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.to_excel(writer,sheet_name='Casos confirmados',index=None)
new_obitos.to_excel(writer,sheet_name='Obitos',index=None)

writer.save()
writer.close()